In [194]:
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
import csv
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision.utils import save_image

import matplotlib.pyplot as plt

In [195]:
H, W = 300, 500

In [196]:
# transform = torchvision.transforms.Resize((H,W))
transform = transforms.Compose([
    transforms.Resize((H,W)),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])
train = torchvision.datasets.ImageFolder("data", transform)
# train = torchvision.datasets.ImageFolder("data", transform)

In [197]:
train_loader = DataLoader(
    train,
    batch_size=4,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

In [198]:
# get waterbirds; returns dict birds
# 1,001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg,1,2,1,/o/ocean/00002178.jpg
# img_id,img_filename,y,split,place,place_filename
def get_place():
    '''
    returns:    birds, dict with keys 
                    '0' for land
                    '1' for water
    '''
    birds = {}
    with open('./waterbirds/metadata.csv') as f:
        reader = csv.DictReader(f)
        for row in reader:
            fname = row['img_filename'].split('/')[-1]
            birds[fname] = int(row['place'])
    return birds
			
# print(get_place())
# '001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg': 1, 
# '001.Black_footed_Albatross/Black_Footed_Albatross_0009_34.jpg': 1,	


In [199]:
birds = get_place()

In [200]:
for batch_idx, (images, labels) in enumerate(train_loader):
      # print("batch", batch_idx)
      # for i, (images, labels) in enumerate(test_loader, 0):
      # outputs = model(images)
      # _, predicted = torch.max(outputs.data, 1)
      fname, _ = train_loader.dataset.samples[batch_idx]
      # print(sample_fname)
      fname = fname.split('/')[-1]
      # print(fname)
      # TODO: handle batch size > 1

In [201]:
class BasicConvNet(nn.Module):
    """This is a small neural network."""

    def __init__(self, num_classes):
        super(BasicConvNet, self).__init__()
        self.conv2d1 = nn.Conv2d(3, 2, 3, 1, 1)
        self.relu = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(2, 2)
        self.conv2d2 = nn.Conv2d(2, 4, 3, 1, 1)
        self.maxpool2 = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(12544, num_classes)
        self._weight_init()

    def _weight_init(self):
        for layer in self.children():
            if (layer.__class__.__name__ == 'Linear' \
                or layer.__class__.__name__ == 'Conv2d'):
                nn.init.kaiming_uniform_(layer.weight.data,0.2)
                nn.init.constant_(layer.bias.data,0)
            

    def forward(self, x):
        x = self.conv2d1(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        x = self.conv2d2(x)
        x = self.relu(x)
        x = self.maxpool2(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.fc(x)
        # x = x.view(-1,1)
        # x = x.view(-1)
        return x

## Train featurizers and classifiers together for all environments

In [211]:
lr = 0.01
n_epochs = 100
birds = get_place()

land_net = BasicConvNet(num_classes=2)
water_net = BasicConvNet(num_classes=2)
land_optimizer = optim.Adam(land_net.parameters(), lr=lr)
water_optimizer = optim.Adam(water_net.parameters(), lr=lr)
water_optimizer.zero_grad()
land_optimizer.zero_grad()
criterion = nn.CrossEntropyLoss()

for epoch in tqdm(range(n_epochs)):
    for batch_idx, data in tqdm(
        enumerate(train_loader), total=len(train_loader)
    ):
        inputs, labels = data

        sample_fname, _ = train_loader.dataset.samples[batch_idx]
        fname = sample_fname.split('/')[-1]

        if birds[fname] == 0:
            # Train land-env birds
            land_optimizer.zero_grad()
            outputs = land_net(inputs)
            land_loss = criterion(outputs, labels.type(torch.LongTensor)) 
            land_loss.backward()
            land_optimizer.step()
            
        elif birds[fname] == 1:
            # Train water-env birds
            water_optimizer.zero_grad()
            outputs = water_net(inputs)
            water_loss = criterion(outputs, labels.type(torch.LongTensor)) 
            water_loss.backward()
            water_optimizer.step()


    print(f"E: {epoch}; Land Loss: {land_loss.item()}; Water Loss: {water_loss.item()}")


  1%|          | 1/100 [00:02<03:46,  2.29s/it]

E: 0; Land Loss: 8.986028671264648; Water Loss: 0.6300394535064697


  2%|▏         | 2/100 [00:04<03:13,  1.98s/it]

E: 1; Land Loss: 0.6716659069061279; Water Loss: 0.6419945359230042


  2%|▏         | 2/100 [00:04<03:22,  2.07s/it]


KeyboardInterrupt: 

## Train classifiers for environments only, freezing featurizers

In [212]:
for layer in land_net.children():
    if layer.__class__.__name__ == 'Linear':
        for p in layer.parameters():
            p.requires_grad = False

for param in water_net.parameters():
    if layer.__class__.__name__ == 'Linear':
        for p in layer.parameters():
            p.requires_grad = False

l_optim_frozen = optim.SGD(filter(lambda p: p.requires_grad, land_net.parameters()), lr=lr)
w_optim_frozen = optim.SGD(filter(lambda p: p.requires_grad, water_net.parameters()), lr=lr)